In [1]:
import gym

from stable_baselines3 import PPO
from stable_baselines3 import SAC
from stable_baselines3 import mSAC
from stable_baselines3.common.evaluation import evaluate_policy, evaluate_meta_policy

import numpy as np
import torch as th
from gym import spaces

##pyfly stuff
from pyfly.pyfly import PyFly
from pyfly.pid_controller import PIDController
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation

In [2]:
th.manual_seed(42)
np.random.seed(666)

In [3]:
import numpy as np
from gym.envs.mujoco import HalfCheetahEnv as HalfCheetahEnv_

class HalfCheetahEnv(HalfCheetahEnv_):
    def _get_obs(self):
        return np.concatenate([
            self.sim.data.qpos.flat[1:],
            self.sim.data.qvel.flat,
            self.get_body_com("torso").flat,
        ]).astype(np.float32).flatten()

    def viewer_setup(self):
        camera_id = self.model.camera_name2id('track')
        self.viewer.cam.type = 2
        self.viewer.cam.fixedcamid = camera_id
        self.viewer.cam.distance = self.model.stat.extent * 0.35
        # Hide the overlay
        self.viewer._hide_overlay = True

    def render(self, mode='human'):
        if mode == 'rgb_array':
            self._get_viewer().render()
            # window size used for old mujoco-py:
            width, height = 500, 500
            data = self._get_viewer().read_pixels(width, height, depth=False)
            return data
        elif mode == 'human':
            self._get_viewer().render()
            
import numpy as np



class HalfCheetahVelEnv(HalfCheetahEnv):
    """Half-cheetah environment with target velocity, as described in [1]. The
    code is adapted from
    https://github.com/cbfinn/maml_rl/blob/9c8e2ebd741cb0c7b8bf2d040c4caeeb8e06cc95/rllab/envs/mujoco/half_cheetah_env_rand.py
    The half-cheetah follows the dynamics from MuJoCo [2], and receives at each
    time step a reward composed of a control cost and a penalty equal to the
    difference between its current velocity and the target velocity. The tasks
    are generated by sampling the target velocities from the uniform
    distribution on [0, 2].
    [1] Chelsea Finn, Pieter Abbeel, Sergey Levine, "Model-Agnostic
        Meta-Learning for Fast Adaptation of Deep Networks", 2017
        (https://arxiv.org/abs/1703.03400)
    [2] Emanuel Todorov, Tom Erez, Yuval Tassa, "MuJoCo: A physics engine for
        model-based control", 2012
        (https://homes.cs.washington.edu/~todorov/papers/TodorovIROS12.pdf)
    """
    def __init__(self, task={}, n_tasks=30000, randomize_tasks=True):
        self._task = task
        self.tasks = self.sample_tasks(n_tasks)
        self._goal_vel = self.tasks[0].get('velocity', 0.0)
        self._goal = self._goal_vel
        self.i = 0
        super(HalfCheetahVelEnv, self).__init__()

    def step(self, action):
        xposbefore = self.sim.data.qpos[0]
        self.do_simulation(action, self.frame_skip)
        xposafter = self.sim.data.qpos[0]

        forward_vel = (xposafter - xposbefore) / self.dt
        forward_reward = -1.0 * abs(forward_vel - self._goal_vel)
        ctrl_cost = 0.5 * 1e-1 * np.sum(np.square(action))

        observation = self._get_obs()
        reward = forward_reward - ctrl_cost
        if self.i >= 200:
            done = True
            self.i = 0
            print(self._goal_vel)
            del self.tasks[0]
            self._goal_vel = self.tasks[0].get('velocity', 0.0)
            self._goal = self._goal_vel
            
        else:
            done = False
            self.i += 1
        infos = dict(reward_forward=forward_reward,
            reward_ctrl=-ctrl_cost, task=self._task)
        return (observation, reward, done, infos)

    def sample_tasks(self, num_tasks):
        np.random.seed(666)
        print('goal sampled')
        velocities = np.random.choice([1.25,1.5], num_tasks)#np.random.uniform(0.0, 3.0, size=(num_tasks,))
        tasks = [{'velocity': velocity} for velocity in velocities]
        return tasks

    def get_all_task_idx(self):
        return range(len(self.tasks))

    def reset_task(self, idx):
        self._task = self.tasks[idx]
        self._goal_vel = self._task['velocity']
        self._goal = self._goal_vel
        self.reset()

In [4]:
env = HalfCheetahVelEnv(n_tasks = 5000000)#gym.make('BipedalWalker-v3')#FooEnv()#

meta_model = mSAC('MlpPolicy', env, verbose=1,policy_kwargs=dict(net_arch=[300, 300, 300], latent_dim = 5, hidden_sizes=[200,200,200]))#,learning_rate=0.0006)

for param in meta_model.actor.context_encoder.parameters():
    print(param.data)

meta_reward = []
meta_std = []

print('-Start-')
n_eval =30

meta_model_mean_reward_before, meta_model_std_reward_before = evaluate_meta_policy(meta_model, env, n_eval_episodes=30, add2buff = True)
meta_reward.append(meta_model_mean_reward_before)
meta_std.append(meta_model_std_reward_before)



print('##################################Start Learning##################################')
for i in range(200):
    
    meta_model.learn(total_timesteps=5*200)#, eval_freq=100, n_eval_episodes=5)
    meta_model_mean_reward, meta_model_std_reward = evaluate_meta_policy(meta_model, env, n_eval_episodes=n_eval)

    meta_reward.append(meta_model_mean_reward)
    meta_std.append(meta_model_std_reward)
    
    
    print('epoch:', i)
    print('meta_reward = ', meta_reward)
    print('meta_std = ', meta_std)
    
env.close()

goal sampled


/home/user/anaconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


Using cpu device
Wrapping the env in a DummyVecEnv.
critic with  31
critic with  31
critic with  31
critic with  31
tensor([[ 0.0537,  0.0913,  0.1016,  ..., -0.1278,  0.1362,  0.1486],
        [-0.0029,  0.1456, -0.1328,  ..., -0.1428,  0.1556,  0.1592],
        [ 0.0502,  0.1107,  0.0090,  ..., -0.0497,  0.0032, -0.1174],
        ...,
        [ 0.0722,  0.0678,  0.0318,  ...,  0.1615, -0.1540,  0.0962],
        [-0.1173, -0.1402, -0.1299,  ...,  0.1444,  0.0846, -0.1017],
        [-0.1145, -0.0638, -0.0062,  ...,  0.1677, -0.0879,  0.1160]])
tensor([ 0.0213, -0.0235, -0.0045,  0.0180,  0.0009,  0.1305,  0.0564,  0.1421,
         0.0740, -0.1379, -0.0046,  0.0697, -0.1318, -0.0349,  0.0828,  0.0944,
         0.1098, -0.0579, -0.1336,  0.1232, -0.0062,  0.0809, -0.0710, -0.0821,
         0.0506, -0.0016, -0.1829, -0.1740, -0.0203,  0.0728,  0.0719, -0.1283,
         0.1463, -0.0662,  0.1097, -0.1368, -0.1147,  0.0393,  0.1382,  0.1829,
        -0.0557,  0.1467,  0.1531,  0.0893, -0.011

collect with prior
1.5
1.25
1.25
-----------------------------
| time/              |      |
|    episodes        | 4    |
|    fps             | 1132 |
|    time_elapsed    | 0    |
|    total timesteps | 804  |
-----------------------------
apply grads
1.25
1.25
1.25
1.25
1.25
1.25
1.25
1.5
1.25
1.5
1.25
1.25
1.5
1.25
1.5
1.5
1.5
1.25
1.25
1.25
1.5
1.25
1.25
1.25
1.25
1.25
1.25
1.5
1.25
1.5
epoch: 0
meta_reward =  [-274.91581210704345, -260.8228147446063]
meta_std =  [25.19121980635234, 52.63247580068557]
rollout
collect with prior
1.5
collect with prior
1.25
1.25
1.5
-------------------------------------
| time/              |              |
|    episodes        | 4            |
|    fps             | 897          |
|    time_elapsed    | 0            |
|    total timesteps | 804          |
| train/             |              |
|    KL_loss         | 3.9373205    |
|    actor_loss      | -0.619       |
|    avg. z          | 0.0009659996 |
|    avg. z var      | 0.43882483   |
|    

collect with prior
1.5
collect with prior
1.25
1.25
1.25
--------------------------------------
| time/              |               |
|    episodes        | 4             |
|    fps             | 965           |
|    time_elapsed    | 0             |
|    total timesteps | 804           |
| train/             |               |
|    KL_loss         | 0.01573646    |
|    actor_loss      | 30.6          |
|    avg. z          | 5.0635448e-05 |
|    avg. z var      | 0.9898409     |
|    critic_loss     | 11.2          |
|    ent_coef        | 0.643         |
|    ent_coef_loss   | -3.95         |
|    learning_rate   | 0.0003        |
|    n_updates       | 1600          |
--------------------------------------
apply grads
1.5
1.5
1.25
1.25
1.5
1.5
1.5
1.5
1.5
1.25
1.25
1.25
1.5
1.5
1.25
1.5
1.5
1.25
1.25
1.25
1.5
1.25
1.5
1.25
1.25
1.25
1.5
1.25
1.5
1.25
epoch: 8
meta_reward =  [-274.91581210704345, -260.8228147446063, -280.9178398885859, -267.1447384518852, -264.6759388804936, -258.19

...

apply grads
1.25
1.5
1.25
1.5
1.25
1.25
1.5
1.25
1.25
1.5
1.25
1.25
1.5
1.25
1.25
1.5
1.25
1.5
1.25
1.5
1.25
1.25
1.25
1.5
1.25
1.25
1.25
1.25
1.25
1.25
epoch: 195
meta_reward =  [-274.91581210704345, -260.8228147446063, -280.9178398885859, -267.1447384518852, -264.6759388804936, -258.19670474889415, -266.5632522717941, -273.1346732879651, -274.32504621616823, -278.32417160854095, -271.8044181260458, -284.07085945080365, -281.7830730813995, -285.3829438469659, -278.9314444333444, -282.2347917231543, -287.640893382687, -287.54471477211337, -285.7561362295773, -278.12707787605797, -289.1571990806973, -278.1233008318172, -283.8094174178629, -279.06234872628255, -279.57373640355297, -289.58992845903, -280.76264409919764, -288.9579400269795, -291.48002402135836, -286.97702395453354, -295.1262330668082, -282.58813543503436, -287.5323365461664, -292.29972384409024, -285.91836933233725, -291.38242196980775, -283.8325901199532, -287.3390759513386, -290.9808941485731, -275.23548189086245, -284.5

apply grads
1.25
1.5
1.5
1.25
1.5
1.5
1.5
1.25
1.25
1.25
1.5
1.5
1.25
1.25
1.5
1.5
1.5
1.25
1.5
1.5
1.5
1.25
1.25
1.25
1.5
1.5
1.5
1.25
1.25
1.25
epoch: 196
meta_reward =  [-274.91581210704345, -260.8228147446063, -280.9178398885859, -267.1447384518852, -264.6759388804936, -258.19670474889415, -266.5632522717941, -273.1346732879651, -274.32504621616823, -278.32417160854095, -271.8044181260458, -284.07085945080365, -281.7830730813995, -285.3829438469659, -278.9314444333444, -282.2347917231543, -287.640893382687, -287.54471477211337, -285.7561362295773, -278.12707787605797, -289.1571990806973, -278.1233008318172, -283.8094174178629, -279.06234872628255, -279.57373640355297, -289.58992845903, -280.76264409919764, -288.9579400269795, -291.48002402135836, -286.97702395453354, -295.1262330668082, -282.58813543503436, -287.5323365461664, -292.29972384409024, -285.91836933233725, -291.38242196980775, -283.8325901199532, -287.3390759513386, -290.9808941485731, -275.23548189086245, -284.59349879

apply grads
1.5
1.5
1.5
1.25
1.5
1.25
1.25
1.25
1.25
1.25
1.25
1.5
1.25
1.25
1.25
1.5
1.25
1.5
1.5
1.5
1.5
1.5
1.5
1.5
1.25
1.5
1.25
1.5
1.5
1.25
epoch: 197
meta_reward =  [-274.91581210704345, -260.8228147446063, -280.9178398885859, -267.1447384518852, -264.6759388804936, -258.19670474889415, -266.5632522717941, -273.1346732879651, -274.32504621616823, -278.32417160854095, -271.8044181260458, -284.07085945080365, -281.7830730813995, -285.3829438469659, -278.9314444333444, -282.2347917231543, -287.640893382687, -287.54471477211337, -285.7561362295773, -278.12707787605797, -289.1571990806973, -278.1233008318172, -283.8094174178629, -279.06234872628255, -279.57373640355297, -289.58992845903, -280.76264409919764, -288.9579400269795, -291.48002402135836, -286.97702395453354, -295.1262330668082, -282.58813543503436, -287.5323365461664, -292.29972384409024, -285.91836933233725, -291.38242196980775, -283.8325901199532, -287.3390759513386, -290.9808941485731, -275.23548189086245, -284.59349879

apply grads
1.25
1.5
1.5
1.5
1.25
1.5
1.5
1.5
1.5
1.5
1.25
1.25
1.5
1.5
1.5
1.5
1.25
1.25
1.5
1.5
1.25
1.25
1.5
1.5
1.25
1.25
1.5
1.5
1.5
1.25
epoch: 198
meta_reward =  [-274.91581210704345, -260.8228147446063, -280.9178398885859, -267.1447384518852, -264.6759388804936, -258.19670474889415, -266.5632522717941, -273.1346732879651, -274.32504621616823, -278.32417160854095, -271.8044181260458, -284.07085945080365, -281.7830730813995, -285.3829438469659, -278.9314444333444, -282.2347917231543, -287.640893382687, -287.54471477211337, -285.7561362295773, -278.12707787605797, -289.1571990806973, -278.1233008318172, -283.8094174178629, -279.06234872628255, -279.57373640355297, -289.58992845903, -280.76264409919764, -288.9579400269795, -291.48002402135836, -286.97702395453354, -295.1262330668082, -282.58813543503436, -287.5323365461664, -292.29972384409024, -285.91836933233725, -291.38242196980775, -283.8325901199532, -287.3390759513386, -290.9808941485731, -275.23548189086245, -284.59349879551

1.25
--------------------------------------
| time/              |               |
|    episodes        | 4             |
|    fps             | 976           |
|    time_elapsed    | 0             |
|    total timesteps | 804           |
| train/             |               |
|    KL_loss         | 0.00017237914 |
|    actor_loss      | 632           |
|    avg. z          | 1.3763234e-06 |
|    avg. z var      | 0.9999461     |
|    critic_loss     | 795           |
|    ent_coef        | 1.19e-05      |
|    ent_coef_loss   | -14.6         |
|    learning_rate   | 0.0003        |
|    n_updates       | 39800         |
--------------------------------------
apply grads
1.5
1.25
1.5
1.5
1.25
1.5
1.25
1.5
1.25
1.5
1.25
1.25
1.5
1.5
1.5
1.25
1.5
1.25
1.5
1.5
1.25
1.5
1.5
1.25
1.5
1.25
1.25
1.25
1.5
1.25
epoch: 199
meta_reward =  [-274.91581210704345, -260.8228147446063, -280.9178398885859, -267.1447384518852, -264.6759388804936, -258.19670474889415, -266.5632522717941, -273.134673287965

In [5]:
# since last time the sampling in evaluation and the sampling depth für context in train

In [6]:
a = [0.7860740250467451, 1.3779506616437, 1.5551784617926114, 0.7858287766954339, 2.928255854632301, 2.1984436580714473, 0.3458226800494484, 1.1588252059030784, 1.885503538619135, 1.3296746059353843, 2.3686750260717098, 2.38235572737476, 2.1685728538042, 2.711781597991056, 1.999393957709451, 1.8307119965196041, 1.260624986955272, 0.2671352016074815, 1.402548298157229, 2.4971121092039477, 2.279859532934837, 2.839511756498454, 0.37517377900679827, 0.5634252162792469, 1.1076871867784575, 0.3762089882646701, 0.9730737106816765, 1.59509821076111, 2.73336720468094, 1.7527743843309642, 1.248311818712994, 1.4736872317976357, 1.3468935565348823, 1.1823408005585798, 2.107571159141275, 2.731734949135858, 0.37990433907251275, 0.7641724093376423, 1.512021117484312, 1.0545722993955051, 1.6782903704242598, 2.8169478016378617, 1.2892761911000037, 1.4989447841160373, 2.992167429662661, 0.47605191646339684, 0.8343795584830792, 0.9630016215605014, 0.5557528352161346, 0.008703090506129718, 0.48863259980930185, 0.5975857056152952, 0.7963510149605818, 0.864501443436107, 0.047374356635443315, 1.2762439954335498, 2.010656574480702, 2.9203781981030765, 2.100895260148338, 2.3680082550723363, 1.6935569384236153, 2.1718436939592927, 2.728960574484791, 0.7084476270060592, 1.480194734343588, 1.022807914959746, 1.9006458178251902, 1.532128187898675, 1.598488390876416, 1.403537916973332, 2.979915119672488, 2.2720545866630646, 0.5357060314905813, 1.474444589494259, 0.48635227727992836, 1.5568759511574917, 2.2805153214175724, 2.0142083586378923, 2.3757763016921394, 2.8720776710137823, 2.195070206002873, 2.079658332603392, 2.9186169896064182, 2.1386625004738615, 0.17603797674033772, 0.5324039546451034, 1.124502553740213, 1.0493076380999455, 2.379069097887579, 0.5713365770889663, 2.9506458154309234, 2.6917428928705496, 0.7125322952456615, 1.0837847172522923, 1.2394239042739765, 1.531412840477841, 1.188193847652982, 2.5079036400976444, 0.043133571081973576, 0.03269028711044775]

In [7]:
len(a)

100

In [8]:
a =[0.7860740250467451, 1.3779506616437, 1.5551784617926114, 0.7858287766954339, 2.928255854632301, 2.1984436580714473, 0.3458226800494484, 1.1588252059030784, 1.885503538619135, 1.3296746059353843, 2.3686750260717098, 2.38235572737476, 2.1685728538042, 2.711781597991056, 2.9784953809599912, 2.096870367728471, 2.04652104928077, 1.999393957709451, 1.8307119965196041, 1.4436558931536259, 1.260624986955272, 1.8408117163039968, 1.0474247818307383, 0.2893499571670589, 0.2671352016074815, 1.402548298157229, 2.4971121092039477, 2.279859532934837, 2.839511756498454, 0.37517377900679827, 0.5634252162792469, 1.1076871867784575, 2.7450762138519322, 2.176699864999294, 0.3762089882646701, 0.9730737106816765, 1.59509821076111, 2.73336720468094, 2.050959316523306, 2.809525172141024, 1.7527743843309642, 1.248311818712994, 1.4736872317976357, 1.3468935565348823, 1.1823408005585798, 2.107571159141275, 2.731734949135858, 0.37990433907251275, 0.7641724093376423, 1.512021117484312, 1.0545722993955051, 1.6782903704242598, 2.8169478016378617, 1.2892761911000037, 0.8861106117834646, 1.4989447841160373, 2.992167429662661, 0.7015287662209706, 0.11168961358780638, 0.47605191646339684, 0.8343795584830792, 0.9630016215605014, 0.5557528352161346, 0.008703090506129718, 0.48863259980930185, 0.5975857056152952, 0.7963510149605818, 0.864501443436107, 0.047374356635443315, 1.2762439954335498, 2.010656574480702, 2.9203781981030765, 2.100895260148338, 2.3680082550723363, 1.6935569384236153, 2.1718436939592927, 2.728960574484791, 0.7084476270060592, 1.480194734343588, 0.09577530004016699, 1.022807914959746, 1.9006458178251902, 1.532128187898675, 1.598488390876416, 1.454276543853031, 1.403537916973332, 1.071058434727445, 2.979915119672488, 2.2720545866630646, 0.5357060314905813, 1.474444589494259, 0.48635227727992836, 1.5568759511574917, 2.2805153214175724, 2.0142083586378923, 2.3757763016921394, 2.2093259883723895, 0.7794378763219836, 2.8720776710137823, 2.195070206002873, 2.079658332603392, 2.9186169896064182, 2.1386625004738615, 2.802629411145565, 0.17603797674033772, 0.5324039546451034, 1.124502553740213, 1.0493076380999455, 1.744195890973208, 0.5583223214918457, 2.379069097887579, 0.08306914545160993, 0.5713365770889663, 1.2224346928609924, 1.1705025304184877, 0.14470529568442736, 2.9506458154309234, 2.6917428928705496, 2.5617488625541105, 2.317384380319182, 0.7125322952456615, 0.3765232265537465, 1.0837847172522923, 1.2394239042739765, 1.531412840477841, 1.188193847652982, 2.5079036400976444, 0.043133571081973576, 0.03269028711044775, 0.7144484446282852]